In [ ]:
# text_chunks_summary

In [1]:
# DIRECTORY SET
import os
import sys
from pathlib import Path
base_dir=Path(os.getcwd()).parent
os.chdir(base_dir)

import yaml
# data path
prompt_yaml_path="destinyapp/prompter_.yaml"

In [2]:
# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import custom modules
from destinyapp import views
from destinyapp.models import TranscriptData
from destinyapp.api import ServerAiFunctions as saf

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

Keys loaded


In [3]:
del sys.modules['destinyapp.api.ServerAiFunctions']
import destinyapp.api.ServerAiFunctions as saf
reload(saf)

async def run_systems(system_functions, input_contexts):
    async def run_system(system_function):
        tasks=[]
        for input_context in input_contexts:
            task=system_function(**input_context)
            tasks.append(task)

        responses=await asyncio.gather(*tasks)
        return responses

    systems=[]
    for system_function in system_functions:
        system=run_system(system_function)
        systems.append(system)
    
    system_responses=await asyncio.gather(*systems)

    return system_responses

# save to prompt yaml
def save_to_prompt_yaml(input_prompt_data):
    with open(prompt_yaml_path, 'r') as file:
        prompt_data=yaml.load(file, Loader=yaml.FullLoader)
    prompt_data["working"]=input_prompt_data
    with open(prompt_yaml_path, 'w') as file:
        yaml.dump(prompt_data, file)

# New vectordb creation and query testing

In [90]:
del sys.modules['destinyapp.api.ServerAiFunctions']
import destinyapp.api.ServerAiFunctions as saf
reload(saf)

<module 'destinyapp.api.ServerAiFunctions' from 'd:\\DestinyFolder\\DestinyRecaps\\DestinyRecapsApi\\serverproject\\destinyapp\\api\\ServerAiFunctions.py'>

In [70]:
transcript_model_data=await saf.grab_transcript_data(video_id="2O8rCcpFswk")

In [5]:
transcript=transcript_model_data.transcript

In [50]:
chunk_size=1000
overlap=500
text_chunks=[transcript[i:i+chunk_size] for i in range(0, len(transcript), (chunk_size-overlap))]

In [51]:
target_batch_size=18
batch_size=len(text_chunks)//((len(text_chunks)//target_batch_size)+1)+1
print(batch_size)
text_batches=[text_chunks[i:i+batch_size] for i in range(0, len(text_chunks), batch_size)]

str_text_batches=[]
for text_batch in text_batches:
    # number the text_v in the string
    str_text_batch=""
    for i, text_v in enumerate(text_batch):
        str_text_batch+=f"{i+1}: {text_v}\n\n"
    str_text_batches.append(str_text_batch)
    



18


In [40]:
print(str_text_batches[0])

0: Destiny: he gets his hat knocked off, big mistake. And we get the boom.

B: We got this guy jumped off the stage. No one attacked the I punched nobod, no one touched him.

C: I don't think that we would endeavor to eliminate all Mexicans if that were to happen. Right?

Destiny: I like how she switched that subtly from Hamas to all Mexicans. Holy baste. So Brianna Joy Grey thinks that every single Palestinian in the Gaza Strip is Hamas? Cause that's the comparison that she just made, right? Right wing conference in Detroit is canceled if the police respond to dispute the leader of controversial right wing group canceled their conference in Detroit, alleging that they were kicked out of the Russell industrial Center in Detroit, where he said they had plans to hold events. The move comes after Detroit police had responded to a dispute involving the group on Friday. This is the Fuentes group. AFPC Nicholas Fuentes of the America first political Action Conference said Saturday on social 

In [52]:
for text_batch in text_batches:
    print(len(text_batch))

18
18
18
18
18
18
11


In [91]:
response_str_and_cost=await saf.vectordb_and_text_chunks_generator.text_chunk_summary_batch(str_text_batches[0])

In [74]:
print(str_text_batches[0])

1: Destiny: he gets his hat knocked off, big mistake. And we get the boom.

B: We got this guy jumped off the stage. No one attacked the I punched nobod, no one touched him.

C: I don't think that we would endeavor to eliminate all Mexicans if that were to happen. Right?

Destiny: I like how she switched that subtly from Hamas to all Mexicans. Holy baste. So Brianna Joy Grey thinks that every single Palestinian in the Gaza Strip is Hamas? Cause that's the comparison that she just made, right? Right wing conference in Detroit is canceled if the police respond to dispute the leader of controversial right wing group canceled their conference in Detroit, alleging that they were kicked out of the Russell industrial Center in Detroit, where he said they had plans to hold events. The move comes after Detroit police had responded to a dispute involving the group on Friday. This is the Fuentes group. AFPC Nicholas Fuentes of the America first political Action Conference said Saturday on social 

In [92]:
print(response_str_and_cost[1])
print(response_str_and_cost[0])

0.0025822500000000003
(1.) Overview: Discussion of an alleged assault incident at a right-wing conference.
Summary: A person's hat was allegedly knocked off during an incident at a right-wing conference, leading to a physical altercation. The details and context of the incident are unclear and debated.

(2.) Overview: Cancellation of a right-wing conference in Detroit due to safety concerns.
Summary: A right-wing conference in Detroit was canceled after the police responded to a dispute involving the organizers, the America First Political Action Conference (AFPAC). The venue cited safety concerns and alleged misrepresentation by the organizers as reasons for the cancellation.

(3.) Overview: Continued discussion of the right-wing conference cancellation in Detroit.
Summary: The AFPAC organizers claimed they had a contract for the venue, but the venue said they were tricked and never had a direct contract with AFPAC. The event was canceled due to concerns about the nature of the event 

In [ ]:
async def generate_text_chunks_summaries(text_chunks):
    async def text_chunk_summary_batch(text_chunk_batch):
        system_functions=[saf.generate_summary]
        input_contexts=[{"text":text_chunk} for text_chunk in text_chunks]

        system_responses=await run_systems(system_functions, input_contexts)

        return system_responses
    
    
    
    text_chunk_batches=[]
    tasks=[]
    for text_chunk_batch in text_chunks:
        text_chunk_batch=text_chunk_summary_batch(text_chunk_batch)
        text_chunk_batches.append(text_chunk_batch)
        tasks.append(text_chunk_summary_batch(text_chunk_batch))
    
    text_chunk_summaries=await asyncio.gather(*tasks)

# Stream Segmentator

In [105]:
# reload
del sys.modules['destinyapp.api.ServerAiFunctions']
import destinyapp.api.ServerAiFunctions as saf
reload(saf)

<module 'destinyapp.api.ServerAiFunctions' from 'd:\\DestinyFolder\\DestinyRecaps\\DestinyRecapsApi\\serverproject\\destinyapp\\api\\ServerAiFunctions.py'>

In [93]:
# load transcript data from video id=hAf0iOS-2V4
transcript_model_data=await saf.grab_transcript_data("hAf0iOS-2V4")



In [94]:
transcript_model_data.meta

'Israel-Hamas conflict: Just war debate, historical roots, AI in warfare, and challenging simplistic narratives\nHere\'s a comprehensive summary of the main topics and smaller details discussed in the collection of summaries:\n\n<h2>Main Topics</h2>\n\n<h3>1. Israel-Hamas Conflict</h3>\n<ul>\n  <li>Israel\'s response to the October 7 attack</li>\n  <li>Debate on whether the conflict can be considered a "just war"</li>\n  <li>Civilian casualties in Gaza and controversy over reported numbers</li>\n  <li>Israel\'s military tactics and policies</li>\n  <li>Hamas\'s tactics, including use of human shields and civilian infrastructure</li>\n  <li>Debate on Israel\'s use of "dumb bombs" vs. precision weapons</li>\n</ul>\n\n<h3>2. Historical Context</h3>\n<ul>\n  <li>Origins of Zionism and Jewish persecution</li>\n  <li>Palestinian displacement and right of return</li>\n  <li>Historical events like the Second Intifada, 1967 war, and Oslo Accords</li>\n  <li>Jewish-Arab relations before the foun

In [106]:
resposne_str, cost, prompt=await saf.stream_visualizer.annotate_segments(transcript_model_data.meta, transcript_model_data.transcript)

In [108]:
print(resposne_str)

Here are the main topics from the recap with their approximate start and end points in the transcript:

1. Israel-Hamas conflict and debate on "just war"

Starts: "B: Wait, just give me the moss."
Ends: "Destiny: International law gives us two good guiding principles for evaluating the justness of a particular conflict."

2. Historical context of the conflict

Starts: "F: But this conflict didn't start on October 7."
Ends: "Destiny: When did Annapolis start? But those talks didn't even. Was that 2007 or 2006?"

3. Political and ideological aspects

Starts: "F: You know, one of the most clear cut examples of identity politics."
Ends: "Destiny: That's a pretty crazy, like maybe not to be jewish, but maybe absent the Holocaust"

4. International law and ethics

Starts: "B: Okay, but that'sry. I'm sorry to interrupt. I just want to get to the bottom of this issue"
Ends: "Destiny: I think. Yeah."

5. Criticism of applying Western frameworks to non-Western conflicts

Starts: "F: State that i

In [109]:
print(transcript_model_data.meta)

Israel-Hamas conflict: Just war debate, historical roots, AI in warfare, and challenging simplistic narratives
Here's a comprehensive summary of the main topics and smaller details discussed in the collection of summaries:

<h2>Main Topics</h2>

<h3>1. Israel-Hamas Conflict</h3>
<ul>
  <li>Israel's response to the October 7 attack</li>
  <li>Debate on whether the conflict can be considered a "just war"</li>
  <li>Civilian casualties in Gaza and controversy over reported numbers</li>
  <li>Israel's military tactics and policies</li>
  <li>Hamas's tactics, including use of human shields and civilian infrastructure</li>
  <li>Debate on Israel's use of "dumb bombs" vs. precision weapons</li>
</ul>

<h3>2. Historical Context</h3>
<ul>
  <li>Origins of Zionism and Jewish persecution</li>
  <li>Palestinian displacement and right of return</li>
  <li>Historical events like the Second Intifada, 1967 war, and Oslo Accords</li>
  <li>Jewish-Arab relations before the founding of Israel</li>
</ul>


In [107]:
for message in prompt:
    print(message["role"]+": "+message["content"])

system: You annotate where segments of a stream are. 
    
The user will give you a recap (like a summary) of a stream and the transcript for that stream.

You must take the recap, go through each item/topic in it, and denote where in the transcript that item begins and ends. To do this you must give the item then about 10 words that start the item/topic and then the 10 words that end the item/topic. 

This will be programmatically parsed to create segments. 
user: This is the recap: Israel-Hamas conflict: Just war debate, historical roots, AI in warfare, and challenging simplistic narratives
Here's a comprehensive summary of the main topics and smaller details discussed in the collection of summaries:

<h2>Main Topics</h2>

<h3>1. Israel-Hamas Conflict</h3>
<ul>
  <li>Israel's response to the October 7 attack</li>
  <li>Debate on whether the conflict can be considered a "just war"</li>
  <li>Civilian casualties in Gaza and controversy over reported numbers</li>
  <li>Israel's military